In [4]:
%pip install transformers
%pip install -U datasets
    #ransformers \
    #atasets  --quiet

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   ------ --------------------------------- 81.9/510.5 kB 2.3 MB/s eta 0:00:01
   ------ --------------------------------- 81.9/510.5 kB 2.3 MB/s eta 0:00:01
   ------- ------------------------------- 92.2/510.5 kB 744.7 kB/s eta 0:00:01
   ---------- --------------------------- 143.4/510.5 kB 853.3 kB/s eta 0:00:01
   -------------- ----------------------- 194.6/510.5 kB 908.0 kB/s eta 0:00:01
   -------------------------------- ------- 409.6/510.5 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 510.5/510.5 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/170.9 kB ? eta -:--:--
   ---------------------------------------- 170.9/170.9 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/24.8 MB 6.9 MB/s eta 0:00:04
    ---

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.2.0 which is incompatible.


In [5]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [9]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
example_indices =[40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)  
    print('Example', i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True)

print('Encoded Sentence:')
print(sentence_encoded["input_ids"][0])
print('Decoded Sentence:')
print(sentence_decoded)

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{outputs}\n')

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Summarize the following conversation
    {dialogue}
    Summary:
    """
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - NO PROMPT ENGINEERING:\n{outputs}\n')

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Summarize the following conversation
    {dialogue}
    
    What was going on?
    """
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

One-Shot Inference

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt =''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt +=f"""
Dialogue:
{dialogue}
What was going on?
{summary}
"""
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt +=f"""
Dialogue:
{dialogue}
What was going on?
"""
    return prompt
    

In [ ]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'INPUT PROMPT:\n{one_shot_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOTS:\n{outputs}')


Few shot inference

In [ ]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOTS:\n{outputs}')
